In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
pd.set_option('display.max_rows', 500)

Extract

In [2]:
area_raw_df = pd.read_csv('./DATA/FAOSTAT_area.csv')
price_raw_df = pd.read_csv('./DATA/FAOSTAT_prices_apple.csv')
yield_raw_df = pd.read_csv('./DATA/FAOSTAT_yield_per_area.csv') #yield per area
total_yield_raw_df = pd.read_csv('./DATA/FAOSTAT_yield_total.csv')
price_raw_df.head()

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code,Item,Year Code,Year,Months Code,Months,Unit,Value,Flag,Flag Description
0,PP,Producer Prices,3,Albania,5532,Producer Price (USD/tonne),515,Apples,1993,1993,7021,Annual value,USD,461.0,NaN,Official data
1,PP,Producer Prices,3,Albania,5532,Producer Price (USD/tonne),515,Apples,1995,1995,7021,Annual value,USD,323.6,NaN,Official data
2,PP,Producer Prices,3,Albania,5532,Producer Price (USD/tonne),515,Apples,1996,1996,7021,Annual value,USD,325.4,NaN,Official data
3,PP,Producer Prices,3,Albania,5532,Producer Price (USD/tonne),515,Apples,1997,1997,7021,Annual value,USD,253.1,NaN,Official data
4,PP,Producer Prices,3,Albania,5532,Producer Price (USD/tonne),515,Apples,1998,1998,7021,Annual value,USD,248.9,NaN,Official data


Transform

In [3]:
area_df = area_raw_df[['Area', 'Year', 'Value']] #(2704, 3)
area_df.columns = area_df.columns.str.replace('Area', 'Country')
area_df.columns = area_df.columns.str.replace('Value', 'Area_ha') #Total area under the crop in the country

price_df = price_raw_df[['Area', 'Year', 'Value']] #(1883, 3)
price_df.columns = price_df.columns.str.replace('Value', 'Price_USD_tonne') #Producer price USD/tonne
price_df.columns = price_df.columns.str.replace('Area', 'Country')
area_df.shape

yield_df = yield_raw_df[['Area', 'Year', 'Value']] #(2679, 3)
yield_df.columns = yield_df.columns.str.replace('Value', 'Yield_hg_ha') #Yield per area hg/ha
yield_df.columns = yield_df.columns.str.replace('Area', 'Country')

total_yield_df = total_yield_raw_df[['Area', 'Year', 'Value']] #(2733, 3)
total_yield_df.columns = total_yield_df.columns.str.replace('Value', 'TotalYield_tonnes') #Total annual yield in tonnes
total_yield_df.columns = total_yield_df.columns.str.replace('Area', 'Country')
total_yield_df.shape

(2733, 3)

Data warehousing

In [4]:
df = area_df.merge(price_df, on=['Country','Year'], how='left')
df = df.merge(yield_df, on=['Country','Year'], how='left')
df = df.merge(total_yield_df, on=['Country','Year'], how='left')
df.head()

,Country,Year,Area_ha,Price_USD_tonne,Yield_hg_ha,TotalYield_tonnes
0,Afghanistan,1991,2346.0,NaN,74625.0,17507.0
1,Afghanistan,1992,2308.0,NaN,74523.0,17200.0
2,Afghanistan,1993,2300.0,NaN,73913.0,17000.0
3,Afghanistan,1994,2350.0,NaN,74468.0,17500.0
4,Afghanistan,1995,2631.0,NaN,74759.0,19669.0


In [ ]:
rows_with_missing_values = df.Price_USD_tonne.isnull()
print(rows_with_missing_values.iloc[0:200])

In [ ]:
df.to_csv('workingdata.csv', index = False)

Check if there are NaN vals in the columns and drop for the primer analysis 

In [ ]:
#dropping Nanrow values for countries where no price was provided
df.dropna(subset = ['Price_USD_tonne'], inplace=True)
#check = df['TotalYield_tonnes'].isnull().values.any()
print(df)

In [ ]:
df_Estonia = df.loc[(df['Country'] == "Estonia") & (df['Year'] != 0)]
df_Estonia.plot(x='Year', y='TotalYield_tonnes', kind='line', title = 'Estonia')   #Example visualized for Estonia
plt.show()

In [15]:
#Sorting the data by country per total yield to identify potential grouping producers categories.
df.sort_values(by = 'TotalYield_tonnes', ascending=False, inplace = True)

# separate the data by years 
years = df.Year.unique()
years = sorted(years)
years_df = {}

for year in years:
    df_name = 'df_' + str(year)
    years_df[df_name] = df.loc[df['Year'] == year]
    years_df[df_name].sort_values(by=['TotalYield_tonnes'], ascending=False, inplace = True)
    
print(years_df.keys())

2679
<class 'numpy.ndarray'>
1854


KeyError: 'Year'

In [8]:
lengths = []
price_lengths = []

for key in years_df.keys():
    length = len(years_df[key])
    lengths.append([key, length])
    
for key in years_price_df.keys():
    length = len(years_price_df[key])
    price_lengths.append([key, length])
    
lengths = pd.DataFrame(lengths, columns = ['year', 'number_of_countries'])
price_lengths = pd.DataFrame(price_lengths, columns = ['year', 'countries_submitted_price'])
lengths = lengths.merge(price_lengths, on='year')
lengths.year = lengths.year.str.extract('(\d+)')
lengths.year=pd.to_numeric(lengths.year)
print(lengths)

    year  number_of_countries  countries_submitted_price
0   1991                   74                         74
1   1992                   92                         92
2   1993                   93                         93
3   1994                   93                         93
4   1995                   93                         93
5   1996                   93                         93
6   1997                   93                         93
7   1998                   93                         93
8   1999                   93                         93
9   2000                   94                         94
10  2001                   94                         94
11  2002                   94                         94
12  2003                   94                         94
13  2004                   94                         94
14  2005                   94                         94
15  2006                   95                         95
16  2007                   94  

In [ ]:
#check how much data is available 

x = np.arange(len(lengths))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, men_means, width, label='Men')
rects2 = ax.bar(x + width/2, women_means, width, label='Women')

#ax.bar(lengths.year, lengths.number_of_countries)
ax.title("Countries Submitted Sata by Years")
ax.set_ylabel('Submitions')
plt.xticks(ticks = lengths.year, rotation=45)
plt.show()

In [ ]:
by_parts = {}

for key, df in years_df.items():
    print(df.describe())
    df

In [ ]:
#devide df by parts for better ploting preformance
by_parts = {}

for key, df in years_df.items():
    df_number_of_parts = math.ceil(len(df) / 20)
    j = 0
    i = 20
    arr = []
    
    for parts in range(df_number_of_parts):
        df_part = df.iloc[j:i]
        arr.append(df_part)
        j += 20
        i += 20
        
    by_parts[key] = arr

print(by_parts['df_1991'][1])
        #plt.bar(df_part["Country"], df_part["TotalYield_tonnes"])

In [ ]:
plt.bar(by_parts['df_2019'][2]['Country'], by_parts['df_2019'][2]["TotalYield_tonnes"])
plt.tick_params(axis="both", direction="in", pad=15)
plt.title("2019")
plt.xticks(rotation='vertical')

plt.show()

In [ ]:
plt.bar(by_parts['df_2019'][1]['Country'], by_parts['df_2019'][1]["TotalYield_tonnes"])
plt.title("2019")
plt.xticks(rotation='vertical')

plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(8,8),nrows=2, ncols=2)
ax1=plt.subplot(2,2,1)

plt.bar(by_parts['df_1993'][0]['Country'], by_parts['df_1993'][0]["TotalYield_tonnes"])
plt.xticks(rotation='vertical')

plt.bar(by_parts['df_1993'][1]['Country'], by_parts['df_1993'][1]["TotalYield_tonnes"])
plt.tick_params(axis="both", direction="in", pad=15)
plt.xticks(rotation='vertical')

plt.bar(by_parts['df_1993'][2]['Country'], by_parts['df_1993'][2]["TotalYield_tonnes"])
plt.tick_params(axis="both", direction="in", pad=15)
plt.xticks(rotation='vertical')

plt.title("1993")
plt.xticks(rotation='vertical')

In [28]:
df_diff = df.copy()
df_diff = df_diff[df_diff.Yield_hg_ha.isna().any()]
print(df_diff)

KeyError: True